In [1]:
import os
import json
import pandas as pd
import random
import pickle
import re

## 데이터 갯수 확인

- 데이터의 불균형이 심하여 데이터 어그맨테이션 필요

In [2]:
# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
    parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

    return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    if len(new_words) != 0:
        sentence = ' '.join(new_words)
        new_words = sentence.split(" ")

    else:
        new_words = ""

    return new_words


def get_synonyms(word):
    synomyms = []

    try:
        for syn in wordnet[word]:
            for s in syn:
                synomyms.append(s)
    except:
        pass

    return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
# def random_insertion(words, n):
# 	return new_words


def add_word(new_words):
    new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_hangul(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word != ""]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4) + 1

    n_sr = max(1, int(alpha_sr*num_words))
    n_ri = max(1, int(alpha_ri*num_words))
    n_rs = max(1, int(alpha_rs*num_words))

#     # sr
#     for _ in range(num_new_per_technique):
#         a_words = synonym_replacement(words, n_sr)
#         augmented_sentences.append(' '.join(a_words))

#     # ri
#     for _ in range(num_new_per_technique):
#         a_words = random_insertion(words, n_ri)
#         augmented_sentences.append(' '.join(a_words))

    # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

#     # rd
#     for _ in range(num_new_per_technique):
#         a_words = random_deletion(words, p_rd)
#         augmented_sentences.append(" ".join(a_words))

    augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

#     augmented_sentences.append(sentence)

    return augmented_sentences

In [3]:
data = pd.read_csv("train.csv", encoding='utf-8', index_col=None)
data = data[['RawText','GeneralPolarity']]

# 새로운 인덱스 설정
data

,RawText,GeneralPolarity
0,가격이 착하고 디자인이 예쁩니다,1
1,시원하게 입을수 있을것 같은데 선물하신분이 너무 크다고 하시는데 반품 비용이 배보...,-1
2,기존32-34사이즈 입습니다 탄력성이좋다해서 30사이즈구매했는데 밴드없는건안들어가고...,-1
3,지난번에 3종 구매하고 굉장히 편하길래 요번에도 믿고 구매했어요. 포인트도 딱 이쁘...,1
4,청바지를 많이 사보긴 했는데 이렇게 스판좋은 청바지는 첨입니다. 둘째낳고 뱃살이 안...,1
...,...,...
1829,아직 사용전입니다 ㅇㅇ 쓰다가 너무 잘찢어져 이걸로 바꿨는데 튼튼해 보여좋네요 때잘...,1
1830,진짜 이걸 왜 이제 샀을까 생각합니다 편리하고 깔끔하게 지워지네요,1
1831,평소 쓰던 제품입니다. 리필은 있는데 막대기가 없어서 막대기 포함되는거 찾아보다가 ...,1
1832,청소기능은 다른 세제기능보다 좋진 않아요. 그러나 편리성 기능은 최고입니다. 이건 ...,0


In [4]:
pos_df = data.loc[data['GeneralPolarity']==1]
neg_df = data.loc[data['GeneralPolarity']== -1]
neu_df = data.loc[data['GeneralPolarity']==0]

len(pos_df), len(neg_df), len(neu_df)

(1202, 270, 362)

## 부정 데이터 증강

In [135]:
# 임시 데이터 프레임
temp_neg_df = pd.DataFrame(columns=["RawText", "GeneralPolarity"])

for t in neg_df['RawText']:
#     print(t)
    aug_data = EDA(t)
    for a_t in aug_data:
#         print(a_t)
        temp_neg_df = pd.concat([temp_neg_df, pd.DataFrame({"RawText": [f"{a_t}"], "GeneralPolarity": [-1]})], ignore_index=True)
#     print()

In [136]:
temp_neg_df

,RawText,GeneralPolarity
0,시원하게 입을수 있을것 같은데 선물하신분이 하시는데 크다고 너무 반품 비용이 배보다...,-1
1,시원하게 입을수 있을것 같은데 입으신다네요. 너무 크다고 하시는데 반품 비용이 배보...,-1
2,시원하게 입을수 배보다 같은데 선물하신분이 너무 크다고 하시는데 반품 비용이 있을것...,-1
3,기존32-34사이즈 입습니다 탄력성이좋다해서 오히려 밴드없는건안들어가고 밴드있는게 ...,-1
4,기존32-34사이즈 입습니다 탄력성이좋다해서 30사이즈구매했는데 밴드없는건안들어가고...,-1
...,...,...
805,핸들 길이조절 고정도 잘안됩니다 그리고 덮개는 힘드네요 이탈해서 걸리적거려요 테이프...,-1
806,핸들 길이조절 고정도 걸리적거려요 그리고 덮개는 툭하면 이탈해서 잘안됩니다 테이프도...,-1
807,제품 실측 사이즈와 조립 후 제가 제 차이가 거 하고 본 납니다. 길어 봉이 너무 ...,-1
808,제품 실측 사이즈와 조립 후 제가 제 본 거 하고 제가 납니다. 차이가 봉이 너무 ...,-1


In [137]:
# 기존 데이터프레임과 합치기
add_neg_df = pd.concat([neg_df, temp_neg_df], ignore_index=True)

# 데이터프레임 내의 행을 무작위로 섞기
add_neg_df = add_neg_df.sample(frac=1, random_state=42).reset_index(drop=True)

add_neg_df

,RawText,GeneralPolarity
0,사이즈는 맞는데 자꾸 벗겨져요 키높이라 바닥이 높아서 그런거 잠깐 한데 같기도 외출...,-1
1,홈쇼핑보다 혹해서 지르고 써본뒤 후회하는게 없고 역시나 후회하는 상품이네요. 깊은 ...,-1
2,"끈적임 구매했는데, 향이 좋다고 더 무슨 설탕물 바른것처럼 끈적이고 개인 취향이겠지...",-1
3,염색약 냄새가 심하네요. 무엇보다 염색 머리 후에 손상이 심해서 안 쓰고 있어요.,-1
4,별로없는듯함 좋은편이나 너무얇아서 차단효과는 신축성은 역한 냄새가 나서 쓰기도 싫음...,-1
...,...,...
1075,딱 얇아서 레이어드해서 입으면좋을듯하여 구매했는데 적립금도쓰고 금액만금이네요 ㆍ 추...,-1
1076,잘 받았어요 제가 발앞쪽 양쪽에 뼈가 좀 튀어나왔는데왼쪽은 괜찮은데 오른쪽 살짝 볼...,-1
1077,향수는 꼭 비싸야 좋은 건 아니라고 하죠. 엄청 저렴한 향수가 있어서 사용해 봤는데...,-1
1078,저렴하게 구매해서 살짝 걱정했었는데 마스크 받아보니 역시나 두께도 많이 얇고 좋네요...,-1


## 중립 데이터 증강

In [138]:
# 임시 데이터 프레임
temp_neu_df = pd.DataFrame(columns=["RawText", "GeneralPolarity"])

for t in neu_df['RawText']:
#     print(t)
    aug_data = EDA(t)
    for a_t in aug_data:
#         print(a_t)
        temp_neu_df = pd.concat([temp_neu_df, pd.DataFrame({"RawText": [f"{a_t}"], "GeneralPolarity": [0]})], ignore_index=True)
#     print()

In [139]:
temp_neu_df

,RawText,GeneralPolarity
0,해요. 핏인줄 알았는데.. 거의 통바지 수준이네요.. 모델이 입은 옷과는 다른 옷이...,0
1,스키니 핏인줄 알았는데.. 거의 통바지 좋아서 모델이 입은 옷과는 다른 옷이란 생각...,0
2,스키니 핏인줄 알았는데.. 거의 통바지 수준이네요.. 편하긴 입은 옷과는 다른 옷이...,0
3,따뜻하고 부드럽습니다ᆢ 원단도 좋구여 안 제가 28입는데ᆢ 27 샀습니다 그래도 큽...,0
4,따뜻하고 부드럽습니다ᆢ 원단도 좋구여 싸이즈는ᆢ 제가 28입는데ᆢ 27 샀습니다 이...,0
...,...,...
1081,흔들림있다는 평 있어서 살짝 걱정했는데 괜찮습니다 않아요 빨래가 많이 들어가지는 생...,0
1082,흔들림있다는 돌려야겠네요. 있어서 살짝 걱정했는데 괜찮습니다 생각보다 빨래가 많이 ...,0
1083,청소기능은 다른 세제기능보다 좋진 않아요. 그러나 편리성 기능은 최고입니다. 이건 ...,0
1084,압축해놓은 마치 세제기능보다 좋진 않아요. 그러나 편리성 기능은 최고입니다. 이건 ...,0


In [140]:
# 기존 데이터프레임과 합치기
add_neu_df = pd.concat([neu_df, temp_neu_df], ignore_index=True)

# 데이터프레임 내의 행을 무작위로 섞기
add_neu_df = add_neu_df.sample(frac=1, random_state=42).reset_index(drop=True)

add_neu_df

,RawText,GeneralPolarity
0,간단한 활용할수있는 디스플레이 제한적이고 메뉴가 변경도 정해진대로만 가능하지만.. ...,0
1,가죽인데. 사이즈도 적당하고 움직이는데도 편하더라고요. 찰 시간이 지나면 땀이 근대...,0
2,이전에도 구입하였었는데 재구매합니다 아직 소음이 걱정없이 합니다만 이전것에 비해 소...,0
3,스판이 편합니다. 있어서 앞으로 입기에는 좀 두꺼운듯 싶어요.,0
4,가격이 굉장히 비싸네요~ 하지만 디자인이 고급스럽고 세련돼서 짱 만족합니다^^,0
...,...,...
1443,카메라 기능도 다양하고 특히 성능 향상이 정말 역대급이라 만족합니다! 건 조금 많이...,0
1444,사이즈가 13인치에 이르니 시원하게 넓어서 배터리가 게임하며 앱을 사용하니 좋아요 ...,0
1445,향이 부드러워 맘에 듭니다. 냄새가 주문했어요. 지속되면 좋겠습니다. 가격은 마트보...,0
1446,제품 제품 작아서 휴대하기 디자인도 좋아요. 공부하는 학생 또는 예뻐서 사용하기 알...,0


## 모든 데이터 합치기

In [142]:
all_data = pd.concat([pos_df, add_neu_df,add_neg_df], ignore_index=True)

# 데이터 섞기
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)

all_data

,RawText,GeneralPolarity
0,"평소에 목주름,팔자주름에 컴플렉스가 심해서 괜찮은 제품 찾던 중이었는데 각종 방송에...",1
1,디자인이 심플하니 예뻐서 마음에 들어요^^ 더 좀 작아서 사이즈가 큰 크기도 나왔으...,0
2,오염에 포스팅이지만 제가 기록을 없음 써봅니다.최근에 구매한 노트북 5. : 물류창...,-1
3,앞판이 좀 칙칙한 색이 왔지만 디자인이 맘에 듭니다..아예 별 기대 안했는데 부드럽...,1
4,우연히 써봤는데 좋아서 구매했습니다 칫솔모가 좀 약한데 전 사이사이 잘 닦이는거 같...,1
...,...,...
3725,ㅇㅇ마스크의 크기가 더욱 넓어져 구입했습니다. 비말이나 바이러스로부터 호흡기를 안전...,1
3726,향이 부드러워 맘에 듭니다. 냄새가 오래 지속되면 좋겠습니다. 가격은 마트보다 조금...,0
3727,이상없이 배송 잘 되었어요. 할인행사중으로 저렴하게 잘 사서 기분 좋아요~ 제품의 ...,1
3728,이미지와 다르게 인식조차 처리가 너무 별로네요 -_- 거기다가 낮은 물체는 마감 못...,-1


In [144]:
# all_data.to_csv("augmentation_data.csv", index=False)

# 임의로 데이터 생성해서 추가해주기

In [1]:
sample='''"이 카드는 혜택이 풍부해서 정말 좋아요." (1)
"카드 신청이 쉽고 빠르게 처리되어서 기분이 좋아졌어요." (1)
"카드 광고가 너무 과장되었고, 실제 혜택은 별로 없어서 실망스러웠어요." (-1)
"신용카드로 할인 혜택을 받아서 좋았지만, 다른 부분에서는 별로였어요." (0)
"카드 사용으로 인한 혜택이 명확하지 않아서 조금 헷갈렸어요." (0)
"매월 적립되는 포인트로 다양한 할인 혜택을 받을 수 있어서 만족스러워요." (1)
"카드 사용으로 인해 생활이 편리해지고, 기분이 좋아졌어요." (1)
"광고에서 언급한 혜택과 실제로 받은 혜택이 일치해서 기분이 좋아졌어요." (1)
"카드 발급에 필요한 서류가 많아서 번거로웠어요." (-1)
"신용카드로 결제하면 항상 할인 혜택을 받아서 좋아요." (1)
"카드 사용으로 인한 혜택은 별로 없어서 실망스러웠어요." (-1)
"카드의 디자인이 마음에 들어서 신청했는데, 기대와 달리 혜택이 부족했어요." (-1)
"카드 사용이 편리하고, 적립된 포인트로 다양한 혜택을 누릴 수 있어서 만족스러워요." (1)
"카드 발급 후 첫 달에 받은 혜택이 기대 이상이어서 기분이 좋아졌어요." (1)
"카드 사용으로 인해 생활이 편리해졌지만, 금리가 조금 아쉬웠어요." (0)
"카드로 할인 혜택을 받아서 좋았지만, 사용처가 제한적이어서 아쉬웠어요." (0)
"카드 발급이 빨리 되어서 좋았어요." (1)
"신용카드 사용으로 인한 혜택이 명확해서 만족스러웠어요." (1)
"광고에서 언급한 혜택과 다르게 실제 혜택이 적어서 실망했어요." (-1)
"카드 사용으로 인한 혜택이 크지 않아서 기대와 달리 실망스러웠어요." (-1)
"카드 사용으로 인해 생활이 편리해지고, 적립된 포인트로 다양한 혜택을 누릴 수 있어서 만족스러워요." (1)
"카드 발급 후 몇 달 사용해봤는데, 특별한 혜택이 없어서 아쉬웠어요." (0)
"카드 사용으로 인해 생활이 편리해지고, 신속한 결제가 가능해서 좋아졌어요." (1)
"카드 발급이 어려워서 신청하는 데에 애를 먹었어요." (-1)
"카드 사용으로 인한 혜택이 명확하고, 친절한 고객 서비스 때문에 만족스러웠어요." (1)
"카드 광고에 언급된 혜택과 실제로 받은 혜택이 일치해서 만족해요." (1)
"카드 사용으로 인해 생활이 편리해지고, 다양한 할인 혜택을 받아서 좋아졌어요." (1)
"신용카드 사용 후 받은 혜택이 예상보다 더 좋아서 기분이 좋아졌어요." (1)
"카드 발급이 빨리 되어서 좋았지만, 금리가 높아서 아쉬웠어요." (0)
"카드 사용으로 인한 혜택이 명확해서 만족스러웠어요." (1)
"카드 광고에 나오는 혜택과 실제로 받은 혜택이 차이가 있어서 실망했어요." (-1)
"카드 사용으로 인해 생활이 편리해졌지만, 일부 가맹점에서 사용이 제한되어서 아쉬웠어요." (0)
"카드 발급이 어려워서 번거로웠어요." (-1)
"카드 사용으로 인한 혜택이 크지 않아서 기대와 달리 실망스러웠어요." (-1)
"카드 사용으로 인해 생활이 편리해지고, 신속한 결제가 가능해서 좋아졌어요." (1)
"카드 발급 후 몇 달 사용해봤는데, 특별한 혜택이 없어서 아쉬웠어요." (0)
"카드 사용으로 인해 생활이 편리해지고, 적립된 포인트로 다양한 혜택을 누릴 수 있어서 만족스러워요." (1)
"카드 발급이 어려워서 신청하는 데에 애를 먹었어요." (-1)
"카드 사용으로 인한 혜택이 명확하고, 친절한 고객 서비스 때문에 만족스러웠어요." (1)
"카드 광고에 언급된 혜택과 실제로 받은 혜택이 일치해서 만족해요." (1)
"카드 사용으로 인해 생활이 편리해지고, 다양한 할인 혜택을 받아서 좋아졌어요." (1)
"신용카드 사용 후 받은 혜택이 예상보다 더 좋아서 기분이 좋아졌어요." (1)
"카드 발급이 빨리 되어서 좋았지만, 금리가 높아서 아쉬웠어요." (0)
"카드 사용으로 인한 혜택이 명확해서 만족스러웠어요." (1)
"카드 광고에 나오는 혜택과 실제로 받은 혜택이 차이가 있어서 실망했어요." (-1)
"카드 사용으로 인해 생활이 편리해졌지만, 일부 가맹점에서 사용이 제한되어서 아쉬웠어요." (0)
"카드 발급이 어려워서 번거로웠어요." (-1)
"카드 사용으로 인한 혜택이 크지 않아서 기대와 달리 실망스러웠어요." (-1)
"카드 사용으로 인해 생활이 편리해지고, 신속한 결제가 가능해서 좋아졌어요." (1)
"카드 발급 후 몇 달 사용해봤는데, 특별한 혜택이 없어서 아쉬웠어요." (0)
"카드 사용으로 인해 생활이 편리해지고, 적립된 포인트로 다양한 혜택을 누릴 수 있어서 만족스러워요." (1)
"카드 발급이 어려워서 신청하는 데에 애를 먹었어요." (-1)
"카드 사용으로 인한 혜택이 명확하고, 친절한 고객 서비스 때문에 만족스러웠어요." (1)
"카드 광고에 언급된 혜택과 실제로 받은 혜택이 일치해서 만족해요." (1)
"카드 사용으로 인해 생활이 편리해지고, 다양한 할인 혜택을 받아서 좋아졌어요." (1)
"신용카드 사용 후 받은 혜택이 예상보다 더 좋아서 기분이 좋아졌어요." (1)
"카드 발급이 빨리 되어서 좋았지만, 금리가 높아서 아쉬웠어요." (0)
"카드 사용으로 인한 혜택이 명확해서 만족스러웠어요." (1)
"카드 광고에 나오는 혜택과 실제로 받은 혜택이 차이가 있어서 실망했어요." (-1)
"카드 사용으로 인해 생활이 편리해졌지만, 일부 가맹점에서 사용이 제한되어서 아쉬웠어요." (0)
"카드 발급이 어려워서 번거로웠어요." (-1)
"카드 사용으로 인한 혜택이 크지 않아서 기대와 달리 실망스러웠어요." (-1)
"카드 사용으로 인해 생활이 편리해지고, 신속한 결제가 가능해서 좋아졌어요." (1)
"카드 발급 후 몇 달 사용해봤는데, 특별한 혜택이 없어서 아쉬웠어요." (0)
"카드 사용으로 인해 생활이 편리해지고, 적립된 포인트로 다양한 혜택을 누릴 수 있어서 만족스러워요." (1)
"카드 발급이 어려워서 신청하는 데에 애를 먹었어요." (-1)
"카드 사용으로 인한 혜택이 명확하고, 친절한 고객 서비스 때문에 만족스러웠어요." (1)
"카드 광고에 언급된 혜택과 실제로 받은 혜택이 일치해서 만족해요." (1)'''

In [4]:
texts = sample.split("\n")

# 임시 데이터 프레임
temp_add_df = pd.DataFrame(columns=["RawText", "GeneralPolarity"])


In [7]:
for text in texts:
    s_text = text.split('"')
    t = " ".join(s_text[:-1])
    pol = int(s_text[-1].replace("(","").replace(")",""))
    print(t, pol)
    for aug_text in EDA(t):
        print(aug_text)
        temp_add_df = pd.concat([temp_add_df, pd.DataFrame({"RawText": [f"{aug_text}"], "GeneralPolarity": f"{pol}"})], ignore_index=True)
    temp_add_df = pd.concat([temp_add_df, pd.DataFrame({"RawText": [f"{t}"], "GeneralPolarity": f"{pol}"})], ignore_index=True)
    print()

 이 카드는 혜택이 풍부해서 정말 좋아요. 1
이 카드는 혜택이 좋아요. 정말 풍부해서
이 정말 혜택이 풍부해서 카드는 좋아요.
이 카드는 좋아요. 풍부해서 정말 혜택이

 카드 신청이 쉽고 빠르게 처리되어서 기분이 좋아졌어요. 1
카드 처리되어서 쉽고 빠르게 신청이 기분이 좋아졌어요.
카드 처리되어서 쉽고 빠르게 신청이 기분이 좋아졌어요.
카드 빠르게 쉽고 신청이 처리되어서 기분이 좋아졌어요.

 카드 광고가 너무 과장되었고, 실제 혜택은 별로 없어서 실망스러웠어요. -1
별로 광고가 너무 과장되었고, 실제 혜택은 카드 없어서 실망스러웠어요.
카드 혜택은 너무 과장되었고, 실제 광고가 별로 없어서 실망스러웠어요.
실제 광고가 너무 과장되었고, 카드 혜택은 별로 없어서 실망스러웠어요.

 신용카드로 할인 혜택을 받아서 좋았지만, 다른 부분에서는 별로였어요. 0
신용카드로 할인 혜택을 부분에서는 좋았지만, 다른 받아서 별로였어요.
신용카드로 받아서 혜택을 할인 좋았지만, 다른 부분에서는 별로였어요.
신용카드로 할인 별로였어요. 받아서 좋았지만, 다른 부분에서는 혜택을

 카드 사용으로 인한 혜택이 명확하지 않아서 조금 헷갈렸어요. 0
카드 헷갈렸어요. 인한 혜택이 명확하지 않아서 조금 사용으로
혜택이 사용으로 인한 카드 명확하지 않아서 조금 헷갈렸어요.
카드 사용으로 조금 혜택이 명확하지 않아서 인한 헷갈렸어요.

 매월 적립되는 포인트로 다양한 할인 혜택을 받을 수 있어서 만족스러워요. 1
다양한 적립되는 포인트로 매월 할인 혜택을 받을 수 있어서 만족스러워요.
매월 적립되는 포인트로 혜택을 할인 다양한 받을 수 있어서 만족스러워요.
매월 적립되는 만족스러워요. 다양한 할인 혜택을 받을 수 있어서 포인트로

 카드 사용으로 인해 생활이 편리해지고, 기분이 좋아졌어요. 1
카드 인해 사용으로 생활이 편리해지고, 기분이 좋아졌어요.
카드 기분이 인해 생활이 편리해지고, 사용으로 좋아졌어요.
사용으로 카드 인해 생활이 편리해지고, 기분이 좋아졌어요.

 광고에서 언

In [16]:
# temp_add_df.to_csv("add_texts.csv", index=False, encoding='utf-8-sig')

In [10]:
temp_add_df

,RawText,GeneralPolarity
0,이 카드는 혜택이 좋아요. 정말 풍부해서,1
1,이 정말 혜택이 풍부해서 카드는 좋아요.,1
2,이 카드는 좋아요. 풍부해서 정말 혜택이,1
3,이 카드는 혜택이 풍부해서 정말 좋아요.,1
4,카드 처리되어서 쉽고 빠르게 신청이 기분이 좋아졌어요.,1
...,...,...
267,"카드 사용으로 인한 혜택이 명확하고, 친절한 고객 서비스 때문에 만족스러웠어요.",1
268,만족해요. 광고에 언급된 혜택과 실제로 받은 혜택이 일치해서 카드,1
269,받은 광고에 언급된 혜택과 실제로 카드 혜택이 일치해서 만족해요.,1
270,카드 광고에 언급된 혜택과 혜택이 받은 실제로 일치해서 만족해요.,1


In [14]:
temp_add_df = temp_add_df.drop_duplicates()

In [15]:
temp_add_df

,RawText,GeneralPolarity
0,이 카드는 혜택이 좋아요. 정말 풍부해서,1
1,이 정말 혜택이 풍부해서 카드는 좋아요.,1
2,이 카드는 좋아요. 풍부해서 정말 혜택이,1
3,이 카드는 혜택이 풍부해서 정말 좋아요.,1
4,카드 처리되어서 쉽고 빠르게 신청이 기분이 좋아졌어요.,1
...,...,...
264,"사용으로 카드 인한 혜택이 명확하고, 친절한 고객 서비스 때문에 만족스러웠어요.",1
265,"카드 사용으로 인한 혜택이 친절한 명확하고, 고객 서비스 때문에 만족스러웠어요.",1
266,"카드 사용으로 혜택이 인한 명확하고, 친절한 고객 서비스 때문에 만족스러웠어요.",1
269,받은 광고에 언급된 혜택과 실제로 카드 혜택이 일치해서 만족해요.,1
